In [2]:
import torch
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os, h5py

from utils import generate_detect_sg, generate_gt_sg
from sgg_benchmark.config.paths_catalog import DatasetCatalog

In [3]:
file_path = '/home/maelic/Documents/PhD/Datasets/GQA/sceneGraphs/train_sceneGraphs.json'
gqa = json.load(open(file_path, 'r'))


In [6]:
print(gqa['2386622'])

{'width': 375, 'objects': {'1284217': {'name': 'tie', 'h': 78, 'relations': [], 'w': 68, 'attributes': ['patterned', 'gold'], 'y': 422, 'x': 183}, '1284223': {'name': 'man', 'h': 474, 'relations': [{'object': '1284224', 'name': 'wearing'}, {'object': '1284220', 'name': 'wearing'}], 'w': 374, 'attributes': ['smiling', 'happy'], 'y': 24, 'x': 1}, '3844336': {'name': 'picture', 'h': 160, 'relations': [{'object': '1284222', 'name': 'on'}, {'object': '1284224', 'name': 'to the left of'}, {'object': '1284225', 'name': 'to the left of'}, {'object': '1284242', 'name': 'to the left of'}, {'object': '1284230', 'name': 'to the left of'}], 'w': 113, 'attributes': [], 'y': 53, 'x': 0}, '1284242': {'name': 'face', 'h': 244, 'relations': [{'object': '1284220', 'name': 'to the right of'}, {'object': '1284240', 'name': 'to the right of'}, {'object': '3844336', 'name': 'to the right of'}, {'object': '1284235', 'name': 'to the right of'}], 'w': 175, 'attributes': [], 'y': 107, 'x': 114}, '1284219': {'nam

# Run Evaluation for test or train split

In [2]:
BASE_PATH="/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/"
SGDET_DIR=BASE_PATH+"checkpoints/VG150/baseline/upload_causal_motif_sgdet"
FASTER_RCNN_DIR=BASE_PATH+"checkpoints/VG150/curated/pretrained_faster_rcnn/model_final.pth"
GLOVE_DIR="/home/maelic/glove"
EVAL_TYPE="test"
CONFIG="configs/VG150/curated/e2e_relation_X_101_32_8_FPN_1x.yaml"
DATASET="VG_stanford_filtered_with_attribute"
TASK="sgdet"

In [39]:
stats = json.load(open('/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/analysis/sg_statistics.json'))

In [ ]:
#!cd .. && pwd && CUDA_VISIBLE_DEVICES=0 python tools/relation_test_net.py --config-file $CONFIG --task  --verbose MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  SOLVER.IMS_PER_BATCH 32 TEST.IMS_PER_BATCH 1 DTYPE "float16" SOLVER.MAX_ITER 30000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 SOLVER.PRE_VAL False GLOVE_DIR $GLOVE_DIR MODEL.PRETRAINED_DETECTOR_CKPT $FASTER_RCNN_DIR OUTPUT_DIR $SGDET_DIR DATASETS.TO_TEST $EVAL_TYPE

In [2]:
!CUDA_VISIBLE_DEVICES=0 python tools/relation_test_net.py --config-file "configs/VG150/curated/e2e_relation_X_101_32_8_FPN_1x.yaml" --task "sgdet" --verbose MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER vctree SOLVER.IMS_PER_BATCH 32 TEST.IMS_PER_BATCH 12 DTYPE "float16" SOLVER.MAX_ITER 30000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 SOLVER.PRE_VAL False GLOVE_DIR /home/maelic/glove MODEL.PRETRAINED_DETECTOR_CKPT /home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/checkpoints/VG150/curated/pretrained_faster_rcnn/model_final.pth OUTPUT_DIR /home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/checkpoints/VG150/curated/sgdet_vctree_tde DATASETS.TO_TEST train

python: can't open file 'tools/relation_test_net.py': [Errno 2] No such file or directory


In [3]:
result_path = SGDET_DIR + '/inference/' + DATASET +'_' + EVAL_TYPE + '/' 
info_dict = json.load(open(result_path + 'visual_info.json'))
detected_result = torch.load(result_path+'eval_results.pytorch')
result_dict = torch.load(result_path + 'result_dict.pytorch')
vg_dict = json.load(open(DatasetCatalog.DATASETS[DATASET]['dict_file']))
vg_data = h5py.File(DatasetCatalog.DATASETS[DATASET]['roidb_file'], 'r')

In [58]:
output_scene_graphs = generate_detect_sg(detected_result, info_dict, vg_dict, 0.5)
pred = [len(pred[0]['relations']) for i, pred in enumerate(output_scene_graphs.values())]
print("Number of total relations predicted: ", sum(pred))

Generating scene graphs :   1%|          | 37/5001 [00:02<05:06, 16.21it/s]


KeyboardInterrupt: 

In [38]:
pred = [len(pred[0]['relations']) for i, pred in enumerate(output_scene_graphs.values())]
print(sum(pred))

7885


In [51]:
input_scene_graphs = generate_gt_sg(vg_data, vg_dict, info_dict)
print(len(input_scene_graphs))
print(len([os.path.basename(img['img_file']).strip('.jpg') for img in info_dict]))

100%|██████████| 5001/5001 [00:00<00:00, 42554.42it/s]

3765
5001


## Compute frequency

In [43]:
def gt_triplets(input_scene_graphs):
    triplets = []
    for graph in tqdm(input_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+vg_dict['idx_to_predicate'][str(r[2])]+'-'+graph[0]['entities'][r[1]]
            triplets.append(rel)
    df = pd.DataFrame(triplets, columns =['Triplet'])
    # count triplet occurences
    df = df.groupby('Triplet').size().reset_index(name='Count')
    # sort by count
    df = df.sort_values(by=['Count'], ascending=False)
    return df
print("Ranking of triplets in the input scene graphs")
out = gt_triplets(input_scene_graphs)
out.head(10)
# total number of triplets
print("Total number of triplets in the input scene graphs: ", out['Count'].sum())

Ranking of triplets in the input scene graphs


100%|██████████| 3765/3765 [00:00<00:00, 369488.16it/s]

Total number of triplets in the input scene graphs:  25259


In [57]:
def pred_triplets(output_scene_graphs):
    triplets = []
    for graph in tqdm(output_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+r[2]+'-'+graph[0]['entities'][r[1]]
            triplets.append(rel)
    df = pd.DataFrame(triplets, columns =['Triplet'])
    # count triplet occurences
    df = df.groupby('Triplet').size().reset_index(name='Count')
    # sort by count
    df = df.sort_values(by=['Count'], ascending=False)
    return df
print("Ranking of triplets in the input scene graphs")
print("Total number of triplets in the input scene graphs: ", out['Count'].sum())
out = pred_triplets(output_scene_graphs)
out.head(10)


Ranking of triplets in the input scene graphs
Total number of triplets in the input scene graphs:  7885


100%|██████████| 5001/5001 [00:00<00:00, 2048609.66it/s]


,Triplet,Count
0,bed-has-pillow,3
41,sink-near-toilet,3
33,pillow-laying on-bed,3
25,horse-has-tail,3
49,toilet-near-sink,3
46,tail-of-horse,3
13,bus-in front of-bus,3
34,pillow-near-pillow,2
35,pizza-above-pizza,2
37,pizza-looking at-pizza,2


## Compute average triplets score

In [6]:
obj_list = ['man', 'woman', 'person']
def triplets_ranking(output_scene_graphs):
    triplets_score = []
    for graph in tqdm(output_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+r[2]+'-'+graph[0]['entities'][r[1]]
            triplets_score.append([rel, r[3]])
    df = pd.DataFrame(triplets_score, columns =['Triplet', 'Score'])
    # average the dataframe by triplet
    df = df.groupby('Triplet')['Score'].mean().reset_index()
    # sort the dataframe by score
    df = df.sort_values(by=['Score'], ascending=False)
    # filter by triplet containing man or woman only
    df = df[df['Triplet'].str.contains('|'.join(obj_list))]
    return df
print(triplets_ranking(output_scene_graphs)[:50])


100%|██████████| 5001/5001 [00:00<00:00, 19759.96it/s]

                       Triplet     Score
5395       person-wearing-boot  0.514324
5336       person-riding-horse  0.474065
4518              man-at-table  0.440927
8625        woman-riding-horse  0.425111
4550              man-has-neck  0.421535
8504            woman-at-table  0.403960
8631         woman-riding-wave  0.396908
8121           vase-behind-man  0.388316
5276       person-eating-pizza  0.385893
4709           man-riding-wave  0.383368
4551              man-has-nose  0.378898
5272           person-at-table  0.375230
4904       mountain-behind-man  0.374231
4541               man-has-eye  0.373814
4542              man-has-face  0.371121
4540               man-has-ear  0.369671
837            book-behind-man  0.368405
4534          man-eating-pizza  0.365307
4838           man-wearing-tie  0.361336
4968               neck-of-man  0.358339
1485        cabinet-behind-man  0.357171
8535         woman-holding-cup  0.353903
8644    woman-sitting on-towel  0.351381
8518            